In [1]:
import pandas as pd

import numpy as np

import warnings

warnings.filterwarnings('ignore')

## Для удобства сгенерим файл с синтетическми данными

In [3]:
n = 345

n = 345

random_days = np.random.randint(1, 32, n)
dates = pd.to_datetime(
    [f"2025-01-{day:02d}" for day in random_days]
)

# Генерация данных
data = {
    "Sum": np.round(np.random.uniform(10, 500, n), 2),
    "CustomerID": np.random.randint(1, 120, n),
    "datePurcase": dates,
    "Product": np.random.choice(
        ["Laptop", "Phone", "Tablet", "Headphones", "Monitor"], n
    )  # случайные товары
}


df = pd.DataFrame(data)
df = df.sort_values(by='datePurcase')

# 5 случайных nan в Sum
nan_idx = np.random.choice(df.index, 5, replace=False)
df.loc[nan_idx, "Sum"] = np.nan

# 7 случайных None в Product
none_idx = np.random.choice(df.index, 7, replace=False)
df.loc[none_idx, "Product"] = None

qty_raw = np.random.lognormal(mean=1, sigma=0.5, size=len(df))
# значения от 1 до 6
qty = np.clip(np.round(qty_raw).astype(int), 1, 6)
df["qty"] = qty



df.to_csv('sample_eda_data.csv', sep='$', index=False)


## Считываем файл и знакомимся с ним визуально

In [4]:
df = pd.read_csv('sample_eda_data.csv', sep='$')
df.head()

,Sum,CustomerID,datePurcase,Product,qty
0,91.19,85,2025-01-01,Tablet,4
1,277.42,31,2025-01-01,Headphones,4
2,280.29,69,2025-01-01,Headphones,1
3,26.49,101,2025-01-01,Tablet,5
4,248.74,118,2025-01-01,Monitor,2


## Сразу переименовыем столбцы по стандарту

In [17]:
df.columns = ['sum', 'customer_id', 'date_purchase', 'product', 'qty']
df.columns

Index(['sum', 'customer_id', 'date_purchase', 'product', 'qty'], dtype='object')

## Cмотрим общую информацию о датафрейме

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sum            340 non-null    float64
 1   customer_id    345 non-null    int64  
 2   date_purchase  345 non-null    object 
 3   product        345 non-null    object 
 4   qty            345 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 13.6+ KB


## Если видим не соответствие значений типу данных осуществляем приведение

In [19]:
df['qty'] = pd.to_numeric(df['qty'])

df['date_purchase'] = pd.to_datetime(df['date_purchase'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   sum            340 non-null    float64       
 1   customer_id    345 non-null    int64         
 2   date_purchase  345 non-null    datetime64[ns]
 3   product        345 non-null    object        
 4   qty            345 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 13.6+ KB


In [21]:
display(df['date_purchase'].max(), df['date_purchase'].min())

Timestamp('2025-01-31 00:00:00')

Timestamp('2025-01-01 00:00:00')

## Смотрим распредление по числовым данным

In [23]:
df[['sum', 'qty']].describe()

,sum,qty
count,340.000000,345.000000
mean,240.503853,3.043478
std,139.647344,1.393869
min,14.680000,1.000000
25%,125.195000,2.000000
50%,240.955000,3.000000
75%,348.267500,4.000000
max,499.860000,6.000000


## Смотрим сумму по группировке

In [10]:
df.groupby('product')['sum'].sum()

product
Headphones    18938.16
Laptop        16545.58
Monitor       12614.15
Phone         17096.01
Tablet        14424.62
Name: sum, dtype: float64

In [11]:
display(df.groupby('customer_id')['sum'].sum().max())
df.groupby('customer_id')['sum'].sum().min()

1990.15

14.73

## Считаем отсутсвующие значения (None, NaN)

In [12]:
display(df['product'].isna().sum())
display(df['sum'].isna().sum())
df.isna().sum()

7

5

sum             5
customer_id     0
date_purcase    0
product         7
qty             0
dtype: int64

## Заменяем пропущенные значения, где имеет смысл и не искажает данные

In [13]:
# df = df.fillna('-1')  - заменит во всем df
df['product'].fillna('Unknown', inplace=True)

## Смотрим уникальные значения по столбцу product 

In [14]:
df['product'].unique()

array(['Tablet', 'Headphones', 'Monitor', 'Laptop', 'Phone', 'Unknown'],
      dtype=object)